In [85]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import numpy as np

In [86]:
# --- PostgreSQL connection config ---
DB_CONFIG = {
    "host": "localhost",       # เปลี่ยนเป็น host ของคุณ
    "port": 5432,
    "dbname": "airflow",
    "user": "airflow",
    "password": "airflow"
}


In [87]:

### Table features emtpy table
# CREATE_TABLES_SQL = [
#     # 1. Train Features
#     """
#     CREATE TABLE IF NOT EXISTS features (
#         feature_date DATE PRIMARY KEY,
#         day_type SMALLINT NOT NULL,
#         dow SMALLINT NOT NULL,
#         rain_average FLOAT,
#         arl FLOAT,
#         bts FLOAT,
#         mrt_blue FLOAT,
#         mrt_purple FLOAT,
#         srt_red FLOAT,
#         mrt_yellow FLOAT,
#         mrt_pink FLOAT
#     );
#     CREATE INDEX IF NOT EXISTS idx_features_date ON features(feature_date);
#     """,
# ]



In [88]:
df = pd.read_csv('../data/augmented_baseline.csv')
df.head()

,วันที่,day_type,holiday_subject,รถไฟฟ้า ARL,รถไฟฟ้า BTS,รถไฟฟ้าสายสีชมพู,รถไฟฟ้าสายสีน้ำเงิน,รถไฟฟ้าสายสีม่วง,รถไฟฟ้าสายสีเหลือง,รถไฟฟ้าสายสีแดง,rain_average,dow
0,2020-01-01,0,วันขึ้นปีใหม่,48790,461400,not available,236053,23135,not available,not available,0.0,2
1,2020-01-02,1,normal,65090,720500,not available,341059,39445,not available,not available,0.0,3
2,2020-01-03,1,normal,69465,782100,not available,373005,43185,not available,not available,0.0,4
3,2020-01-04,0,normal,53304,574400,not available,274151,30550,not available,not available,0.0,5
4,2020-01-05,0,normal,50992,503000,not available,237162,26012,not available,not available,0.0,6


In [89]:
# --- หลังจากอ่าน df มาแล้ว ให้ตรวจสอบชื่อคอลัมน์ก่อน ---
print(df.columns)

# --- ถ้าชื่อไม่ตรง ให้ map/rename ก่อน เช่น ---
rename_map = {
    "วันที่":       "feature_date",
    "dow":    "dow",
    "day_type":   "day_type",
    "rain_average":   "rain_average",
    "รถไฟฟ้า ARL":   "arl",
    "รถไฟฟ้า BTS":   "bts",
    "รถไฟฟ้าสายสีน้ำเงิน":   "mrt_blue",
    "รถไฟฟ้าสายสีม่วง":   "mrt_purple",
    "รถไฟฟ้าสายสีชมพู":   "mrt_pink",
    "รถไฟฟ้าสายสีแดง":   "srt_red",
    "รถไฟฟ้าสายสีเหลือง":   "mrt_yellow",
    # ... เพิ่ม mapping ตามจริงใน df ของคุณ ...
}
df = df.rename(columns=rename_map)

df

Index(['วันที่', 'day_type', 'holiday_subject', 'รถไฟฟ้า ARL', 'รถไฟฟ้า BTS',
       'รถไฟฟ้าสายสีชมพู', 'รถไฟฟ้าสายสีน้ำเงิน', 'รถไฟฟ้าสายสีม่วง',
       'รถไฟฟ้าสายสีเหลือง', 'รถไฟฟ้าสายสีแดง', 'rain_average', 'dow'],
      dtype='object')


,feature_date,day_type,holiday_subject,arl,bts,mrt_pink,mrt_blue,mrt_purple,mrt_yellow,srt_red,rain_average,dow
0,2020-01-01,0,วันขึ้นปีใหม่,48790,461400,not available,236053,23135,not available,not available,0.000000,2
1,2020-01-02,1,normal,65090,720500,not available,341059,39445,not available,not available,0.000000,3
2,2020-01-03,1,normal,69465,782100,not available,373005,43185,not available,not available,0.000000,4
3,2020-01-04,0,normal,53304,574400,not available,274151,30550,not available,not available,0.000000,5
4,2020-01-05,0,normal,50992,503000,not available,237162,26012,not available,not available,0.000000,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2068,2025-08-30,0,normal,55827,630745,53569,342903,50692,38033,30881,0.000000,5
2069,2025-08-31,0,normal,51281,537471,45400,286080,41090,34682,28622,3.733333,6
2070,2025-09-01,1,normal,74607,814960,75159,488170,84815,50087,43615,16.722222,0
2071,2025-09-02,1,normal,73273,831023,75584,510420,86519,50517,41083,4.944444,1


In [90]:
df[365:]

,feature_date,day_type,holiday_subject,arl,bts,mrt_pink,mrt_blue,mrt_purple,mrt_yellow,srt_red,rain_average,dow
365,2020-12-31,0,วันสิ้นปี,17262,231790,not available,86163,11394,not available,not available,0.000000,3
366,2021-01-01,0,วันขึ้นปีใหม่,14666,181433,not available,82015,9381,not available,not available,0.000000,4
367,2021-01-02,0,normal,14774,178983,not available,76758,10062,not available,not available,0.000000,5
368,2021-01-03,0,normal,17287,180283,not available,76167,10092,not available,not available,0.000000,6
369,2021-01-04,1,normal,30917,376301,not available,178984,25528,not available,not available,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2068,2025-08-30,0,normal,55827,630745,53569,342903,50692,38033,30881,0.000000,5
2069,2025-08-31,0,normal,51281,537471,45400,286080,41090,34682,28622,3.733333,6
2070,2025-09-01,1,normal,74607,814960,75159,488170,84815,50087,43615,16.722222,0
2071,2025-09-02,1,normal,73273,831023,75584,510420,86519,50517,41083,4.944444,1


In [91]:
df.columns

Index(['feature_date', 'day_type', 'holiday_subject', 'arl', 'bts', 'mrt_pink',
       'mrt_blue', 'mrt_purple', 'mrt_yellow', 'srt_red', 'rain_average',
       'dow'],
      dtype='object')

In [92]:
df.replace(['not available'], np.nan, inplace=True)

In [93]:
df

,feature_date,day_type,holiday_subject,arl,bts,mrt_pink,mrt_blue,mrt_purple,mrt_yellow,srt_red,rain_average,dow
0,2020-01-01,0,วันขึ้นปีใหม่,48790,461400,NaN,236053,23135,NaN,NaN,0.000000,2
1,2020-01-02,1,normal,65090,720500,NaN,341059,39445,NaN,NaN,0.000000,3
2,2020-01-03,1,normal,69465,782100,NaN,373005,43185,NaN,NaN,0.000000,4
3,2020-01-04,0,normal,53304,574400,NaN,274151,30550,NaN,NaN,0.000000,5
4,2020-01-05,0,normal,50992,503000,NaN,237162,26012,NaN,NaN,0.000000,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2068,2025-08-30,0,normal,55827,630745,53569,342903,50692,38033,30881,0.000000,5
2069,2025-08-31,0,normal,51281,537471,45400,286080,41090,34682,28622,3.733333,6
2070,2025-09-01,1,normal,74607,814960,75159,488170,84815,50087,43615,16.722222,0
2071,2025-09-02,1,normal,73273,831023,75584,510420,86519,50517,41083,4.944444,1


In [94]:
#CODE for csv to features table in postgresql
from io import StringIO

# เชื่อมต่อฐานข้อมูล
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

# --- จัดเรียงคอลัมน์ให้ตรงกับลำดับในตาราง ---
cols = [
    "feature_date", "day_type", "dow", "rain_average",
    "arl", "bts", "mrt_blue", "mrt_purple",
    "srt_red", "mrt_yellow", "mrt_pink"
]
df = df[cols]

# --- ต่อด้วยการสร้าง buffer และ COPY ตามเดิม ---
from io import StringIO
buffer = StringIO()
df.to_csv(buffer, sep="\t", header=False, index=False)
buffer.seek(0)
cur.copy_from(buffer, "features", sep="\t", null="")
conn.commit()

In [95]:
# ตรวจสอบข้อมูลล่าสุดที่เพิ่งอัปเดต
conn = psycopg2.connect(**DB_CONFIG)

# ตรวจสอบข้อมูลใหม่ตั้งแต่เดือนกันยายน 2025
query = """
SELECT feature_date, day_type, dow, rain_average, arl, bts, mrt_blue, mrt_purple, srt_red, mrt_yellow, mrt_pink 
FROM features 
WHERE feature_date >= '2025-09-04' 
ORDER BY feature_date 
LIMIT 10;
"""

df_new = pd.read_sql(query, conn)
print("📊 ข้อมูลใหม่ที่เพิ่งอัปเดต:")
print(df_new)
conn.close()

📊 ข้อมูลใหม่ที่เพิ่งอัปเดต:
Empty DataFrame
Columns: [feature_date, day_type, dow, rain_average, arl, bts, mrt_blue, mrt_purple, srt_red, mrt_yellow, mrt_pink]
Index: []


C:\Users\ASUS\AppData\Local\Temp\ipykernel_5404\3625735616.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_new = pd.read_sql(query, conn)


In [96]:
# ตรวจสอบสถิติข้อมูลทั้งหมด
conn = psycopg2.connect(**DB_CONFIG)

summary_query = """
SELECT 
    COUNT(*) as total_records,
    MIN(feature_date) as earliest_date,
    MAX(feature_date) as latest_date,
    COUNT(CASE WHEN day_type = -1 THEN 1 END) as pending_day_type,
    COUNT(CASE WHEN rain_average IS NULL THEN 1 END) as missing_rain,
    AVG(arl) as avg_arl,
    AVG(bts) as avg_bts
FROM features;
"""

summary_df = pd.read_sql(summary_query, conn)
print("📈 สถิติข้อมูลทั้งหมด:")
print(summary_df)

# ตรวจสอบข้อมูลที่ต้องแก้ไข day_type และ rain_average
pending_query = """
SELECT 
    EXTRACT(year FROM feature_date) as year,
    COUNT(*) as records_with_pending_data
FROM features 
WHERE day_type = -1 OR rain_average IS NULL
GROUP BY EXTRACT(year FROM feature_date)
ORDER BY year;
"""

pending_df = pd.read_sql(pending_query, conn)
print("\n⚠️ ข้อมูลที่ยังต้องแก้ไข (day_type = -1 หรือ rain_average = null):")
print(pending_df)

conn.close()

📈 สถิติข้อมูลทั้งหมด:
   total_records earliest_date latest_date  pending_day_type  missing_rain  \
0           2073    2020-01-01  2025-09-03                 0             0   

        avg_arl        avg_bts  
0  48990.689822  576819.667631  

⚠️ ข้อมูลที่ยังต้องแก้ไข (day_type = -1 หรือ rain_average = null):
Empty DataFrame
Columns: [year, records_with_pending_data]
Index: []


C:\Users\ASUS\AppData\Local\Temp\ipykernel_5404\652590329.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  summary_df = pd.read_sql(summary_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5404\652590329.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pending_df = pd.read_sql(pending_query, conn)


## สรุปผลการอัปเดตข้อมูล MOT

✅ **สำเร็จ**: ระบบ `update_latest_data.py` ทำงานได้สมบูรณ์
- ดึงข้อมูลใหม่ 27 รายการจาก MOT API (วันที่ 2025-09-04 ถึง 2025-09-30)
- ใช้ UPSERT เพื่อป้องกันข้อมูลซ้ำ
- Mapping ชื่อยานพาหนะภาษาไทยไปเป็น column ภาษาอังกฤษถูกต้อง

📝 **การทำงานของ Mapping**:
```python
vehicle_mapping = {
    'รถไฟฟ้า ARL': 'arl',           # จาก API -> DB
    'รถไฟฟ้า BTS': 'bts', 
    'รถไฟฟ้าสายสีน้ำเงิน': 'mrt_blue',
    'รถไฟฟ้าสายสีม่วง': 'mrt_purple',
    'รถไฟฟ้าสายสีชมพู': 'mrt_pink',
    'รถไฟฟ้าสายสีแดง': 'srt_red',
    'รถไฟฟ้าสายสีเหลือง': 'mrt_yellow'
}
```

⚠️ **สิ่งที่ต้องดำเนินการต่อ**:
- `day_type = -1` (ต้องคำนวณเป็น 0=วันธรรมดา, 1=วันหยุด)
- `rain_average = null` (ต้องเติมข้อมูลฝน)